In [1]:
import matplotlib.pyplot as plt
# %matplotlib inline

plt.style.use('ggplot')
import mne
import pandas as pd
from mne.channels.montage import get_builtin_montages
import numpy as np
import sys
from datetime import time
from datetime import datetime 

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from mne.time_frequency import psd_array_welch
from mne.time_frequency import psd_welch



import mne
from mne.datasets.sleep_physionet.age import fetch_data
from mne.time_frequency import psd_welch


from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, precision_recall_curve, auc, make_scorer, recall_score, accuracy_score, precision_score, confusion_matrix

import matplotlib.pyplot as plt
plt.style.use("ggplot")

from mne import io, EvokedArray
from mne.decoding import Vectorizer, get_coef
from mne.decoding import SlidingEstimator, cross_val_multiscore

from imblearn.ensemble import BalancedRandomForestClassifier


from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample, shuffle

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, recall_score, precision_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score, cross_validate



# import a linear classifier from mne.decoding
from mne.decoding import LinearModel

Using TensorFlow backend.
//anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
//anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
//anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
//anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: Future

In [3]:
def get_records_lists():
    
    records_loc_lst = []
    with open('chb-mit-scalp-eeg-database-1.0.0/RECORDS') as f:
        line_list = f.read().splitlines()
        for elem in line_list:
            if len(elem) > 1:
                records_loc_lst.append(elem)
            
            
    records_w_seizures = []
    with open('chb-mit-scalp-eeg-database-1.0.0/RECORDS-WITH-SEIZURES') as f:
        line_list = f.readlines()
        for elem in line_list:
            if len(elem) > 1:
                records_w_seizures.append(elem.split("/")[1].split('.')[0])
    records_w_seizures[34] = 'chb07_19'            

    records_lst = []
    for rec in records_loc_lst:
        rec_name = rec.split('/')[1].split('.')[0]
        records_lst.append(rec_name)

    return    records_loc_lst, records_w_seizures, records_lst 

In [4]:
recs_l, rec_wsl, rec_wos = get_records_lists()

In [5]:
def get_seizures_dictionary():

    d = {}
    with open('seizures.txt') as f:
        line_list = f.readlines()
        for elem in line_list:
            if elem[0:9] == 'File Name':
                elem2 = elem.split(":")[1]
                fn = elem2.split(".")[0].strip()
                d[fn]={}
                
            if elem[0:26] == 'Number of Seizures in File':
                numb = int(elem.split(":")[1])
                d[fn]['Number of Seizures in File'] = numb
                
            
            if (elem[0:18]   ==  'Seizure Start Time') or (elem[0:20]   ==  'Seizure 1 Start Time'):
                    elem2 = elem.split(":")[1]
                    elem3 = int(elem2.split()[0])
                    d[fn]['Seizure 1 Start Time'] = elem3


            if (elem[0:16]   ==  'Seizure End Time') or (elem[0:18]   ==  'Seizure 1 End Time'):
                    elem2 = elem.split(":")[1]
                    elem3 = int(elem2.split()[0])
                    d[fn]['Seizure 1 End Time'] = elem3
                    d[fn]['Seizure 1 Duration'] = elem3 - d[fn]['Seizure 1 Start Time']    


            if elem[0:20]   ==  'Seizure 2 Start Time':
                elem2 = elem.split(":")[1]
                elem3 = int(elem2.split()[0])
                d[fn]['Seizure 2 Start Time'] = elem3


            if elem[0:18]   ==  'Seizure 2 End Time':
                elem2 = elem.split(":")[1]
                elem3 = int(elem2.split()[0])
                d[fn]['Seizure 2 End Time'] = elem3
                d[fn]['Seizure 2 Duration'] = elem3 - d[fn]['Seizure 2 Start Time']  

            if elem[0:20]   ==  'Seizure 3 Start Time':
                elem2 = elem.split(":")[1]
                elem3 = int(elem2.split()[0])
                d[fn]['Seizure 3 Start Time'] = elem3


            if elem[0:18]   ==  'Seizure 3 End Time':
                elem2 = elem.split(":")[1]
                elem3 = int(elem2.split()[0])
                d[fn]['Seizure 3 End Time'] = elem3
                d[fn]['Seizure 3 Duration'] = elem3 - d[fn]['Seizure 3 Start Time']  

            if elem[0:20]   ==  'Seizure 4 Start Time':
                elem2 = elem.split(":")[1]
                elem3 = int(elem2.split()[0])
                d[fn]['Seizure 4 Start Time'] = elem3


            if elem[0:18]   ==  'Seizure 4 End Time':
                elem2 = elem.split(":")[1]
                elem3 = int(elem2.split()[0])
                d[fn]['Seizure 4 End Time'] = elem3
                d[fn]['Seizure 4 Duration'] = elem3 - d[fn]['Seizure 4 Start Time']   


            if elem[0:20]   ==  'Seizure 5 Start Time':
                elem2 = elem.split(":")[1]
                elem3 = int(elem2.split()[0])
                d[fn]['Seizure 5 Start Time'] = elem3


            if elem[0:18]   ==  'Seizure 5 End Time':
                elem2 = elem.split(":")[1]
                elem3 = int(elem2.split()[0])
                d[fn]['Seizure 5 End Time'] = elem3
                d[fn]['Seizure 5 Duration'] = elem3 - d[fn]['Seizure 5 Start Time'] 
            
    return d 

In [6]:
d = get_seizures_dictionary()

In [7]:
def make_annotations_dict(d):
    import mne

    anno_nm_lst = [n for n in range(138)]
    anno_lst = [n for n in range(138)]
    idx = 0
    for elem in d.keys():
        
        df = d[elem]
        numb = df[list(df)[0]]
        duration = [df[list(df)[i]] for i in range(3, df[list(df)[0]]*3 + 1, 3)]
        onset = [df[list(df)[i]] for i in range(1, 3*numb + 1, 3)]
        description1=['Seizure']*numb
        onset3 = [0.0]
        onset2 = [onset[i] + duration[i] for i in range(numb)]
        onset2 = onset3 + onset2
        duree = [onset[i] - onset2[i] for i in range(numb)]    

        duration2 = [df[list(df)[-1]] - onset2[-1]]
        
        
        description2 = ['Nonseizure']*(numb + 1)
        onset = onset2 + onset
        duration = duree + duration2 + duration
        
        description = description2 + description1

        
        anno = elem 
        
        anno_nm_lst[idx] = anno
        anno = mne.Annotations(onset=onset,
                                duration=duration,
                                description=description)

        
        
        anno_lst[idx] = anno
        idx += 1

    annotations_dict = {anno_nm_lst[i]: anno_lst[i] for i in range(138)}

    return annotations_dict

In [8]:
annotations_dict = make_annotations_dict(d)

In [8]:
def transform_raw_data_channelchanges(subject='chb03', annotations_dict=annotations_dict):
    import numpy as np
    import mne
    from mne.channels.montage import get_builtin_montages
    fif_lst = []
    for rec in recs_l:

        if rec[0:5] == subject:
                data_path = ''
                fname = data_path + rec
                data = mne.io.read_raw_edf(fname, preload=True)
                data = data.filter(1., 40., fir_design='firwin', n_jobs=1)
                info = data.info
                
#                 info['ch_names'] = ['AF7', 'FT7', 'TP7', 'PO7', 'AF3', 'FC3', 'CP3', 'PO3', 'AF4', 'FC4', 'CP4', 'PO4', 'AF8', 'FT8', 'TP8', 'PO8', 'FCz', 'CPz', 'T7', 'FT9', 'FT10', 'T8', 'TP8-1']
#                 numbs1 = [1, 2, 3, 4, 6, 7, 8, 9, 11, 12, 14, 15, 16, 17, 19, 20, 21, 22, 24, 25, 26, 27, 28] 

#                 numbs1n = [num-1 for num in numbs1]
                
#                 for j in range(len(numbs1n)):
#                     info['chs'][numbs1n[j]]['ch_name'] = info['ch_names'][j]
                    
#                 montage = mne.channels.read_montage("standard_1020")
#                 data.set_montage(montage)    
                    
                rec_name = rec.split('/')[1].split('.')[0]
                if rec_name in annotations_dict:
                    anno = annotations_dict[rec_name]
                    data.set_annotations(anno)
            
                    event_id = dict(Seizure=1, Nonseizure=0)
                    events_from_annot, event_dict = mne.events_from_annotations(data, chunk_duration=1)
                    
                    
                if rec_name not in annotations_dict:
                    events = mne.make_fixed_length_events(data, id=0, start=0, stop=None, duration=1.0, first_samp=True, overlap=0.0)
                    
                    
                    mapping = {0: 'Nonseizure', 1: 'Seizure'}
                    onsets = events[:, 0] / data.info['sfreq']
                    durations = np.ones_like(onsets)  # assumes instantaneous events
                    descriptions = [mapping[event_id] for event_id in events[:, 2]]
                    annot_from_events = mne.Annotations(onset=onsets, duration=durations,
                                                        description=descriptions)
                    data.set_annotations(annot_from_events)    
                    
                
                
                
        
                fif = '_raw.fif'
                file_nm = rec_name + fif
                fif_lst.append(file_nm)
                file_nm = 'fif' + subject[3:5] + '/' + file_nm
                data.save(file_nm, fmt='single')
        
    return fif_lst 

In [24]:
transform_raw_data_channelchanges(subject='chb21')
    

Extracting EDF parameters from /home/ubuntu/chb21/chb21_01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 845 samples (3.301 sec)



<ipython-input-23-9b3b3297d477>:11: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(fname, preload=True)
<ipython-input-23-9b3b3297d477>:11: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data = mne.io.read_raw_edf(fname, preload=True)


Writing /home/ubuntu/fif21/chb21_01_raw.fif
Closing /home/ubuntu/fif21/chb21_01_raw.fif [done]
Extracting EDF parameters from /home/ubuntu/chb21/chb21_02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 845 samples (3.301 sec)



<ipython-input-23-9b3b3297d477>:11: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(fname, preload=True)
<ipython-input-23-9b3b3297d477>:11: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data = mne.io.read_raw_edf(fname, preload=True)


Writing /home/ubuntu/fif21/chb21_02_raw.fif
Closing /home/ubuntu/fif21/chb21_02_raw.fif [done]
Extracting EDF parameters from /home/ubuntu/chb21/chb21_03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 845 samples (3.301 sec)



...

<ipython-input-23-9b3b3297d477>:11: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(fname, preload=True)
<ipython-input-23-9b3b3297d477>:11: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data = mne.io.read_raw_edf(fname, preload=True)


Writing /home/ubuntu/fif21/chb21_32_raw.fif
Closing /home/ubuntu/fif21/chb21_32_raw.fif [done]
Extracting EDF parameters from /home/ubuntu/chb21/chb21_33.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 845 samples (3.301 sec)



<ipython-input-23-9b3b3297d477>:11: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8', '-'}. Applying running numbers for duplicates.
  data = mne.io.read_raw_edf(fname, preload=True)
<ipython-input-23-9b3b3297d477>:11: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  data = mne.io.read_raw_edf(fname, preload=True)


Writing /home/ubuntu/fif21/chb21_33_raw.fif
Closing /home/ubuntu/fif21/chb21_33_raw.fif [done]


['chb21_01_raw.fif',
 'chb21_02_raw.fif',
 'chb21_03_raw.fif',
 'chb21_04_raw.fif',
 'chb21_05_raw.fif',
 'chb21_06_raw.fif',
 'chb21_07_raw.fif',
 'chb21_08_raw.fif',
 'chb21_09_raw.fif',
 'chb21_10_raw.fif',
 'chb21_11_raw.fif',
 'chb21_12_raw.fif',
 'chb21_13_raw.fif',
 'chb21_14_raw.fif',
 'chb21_15_raw.fif',
 'chb21_16_raw.fif',
 'chb21_17_raw.fif',
 'chb21_18_raw.fif',
 'chb21_19_raw.fif',
 'chb21_20_raw.fif',
 'chb21_21_raw.fif',
 'chb21_22_raw.fif',
 'chb21_23_raw.fif',
 'chb21_24_raw.fif',
 'chb21_25_raw.fif',
 'chb21_26_raw.fif',
 'chb21_27_raw.fif',
 'chb21_28_raw.fif',
 'chb21_29_raw.fif',
 'chb21_30_raw.fif',
 'chb21_31_raw.fif',
 'chb21_32_raw.fif',
 'chb21_33_raw.fif']

In [9]:
fif_lst = ['fif' + rec_name[3:5] + '/' + rec_name + '_raw.fif' for rec_name in rec_wos if (rec_name[0:5] == 'chb01')]

# or rec_name[0:5] == 'chb02' or rec_name[0:5] == 'chb09' or rec_name[0:5] == 'chb03' or rec_name[0:5] == 'chb07')]

In [10]:
raws = mne.concatenate_raws([mne.io.read_raw_fif(f, preload=False) for f in fif_lst])
info = raws.info
channels = raws.ch_names

Opening raw data file fif01/chb01_01_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif01/chb01_02_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif01/chb01_03_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif01/chb01_04_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif01/chb01_05_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif01/chb01_06_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif01/chb01_07_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif01/chb01_08_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif01/chb01_09_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs

In [11]:

event_id = dict(Seizure=1, Nonseizure=0)
events_from_annot, event_dict = mne.events_from_annotations(raws, event_id=event_id, chunk_duration=1)

Used Annotations descriptions: ['Nonseizure', 'Seizure']


In [12]:
events_from_annot, np.count_nonzero(events_from_annot, axis=0)

(array([[       0,        0,        0],
        [     256,        0,        0],
        [     512,        0,        0],
        ...,
        [37372160,        0,        0],
        [37372416,        0,        0],
        [37372672,        0,        0]]), array([145987,      0,    442]))

In [13]:
tmin, tmax = -.5, .5
epochs = mne.Epochs(raws, events_from_annot, event_id, tmin, tmax, proj=True,
                    decim=2, baseline=None, preload=True)

labels = epochs.events[:, -1]

# get MEG and EEG data

picks = mne.pick_types(epochs.info, meg=False, eeg=True)
X9 = epochs.get_data()[:, picks]
y9 = epochs.events[:, 2]
X9.shape, y9.shape

X9_2d = X9.reshape(len(X9), -1)

from sklearn.model_selection import train_test_split
X9_train, X9_test, y9_train, y9_test = train_test_split(
X9_2d, y9, stratify=y9)

# concatenate our training data back together
X9_stack = np.column_stack((X9_train, y9_train))

# separate minority and majority classes
a = y9_train == 1
seizures = X9_stack[a,:]
nonseizures = X9_stack[~a,:]
nonseizures_downsampled = resample(nonseizures,
                                replace = False, # sample without replacement
                                n_samples = int(10*len(seizures)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled9 = np.vstack((seizures, nonseizures_downsampled))
shuff_down9 = shuffle(downsampled9)

y9_down = shuff_down9[:,-1]
X9_down = np.delete(shuff_down9, -1, axis=1)

clf9 = BalancedRandomForestClassifier(n_estimators=300, max_depth=3, random_state=0)
clf9.fit(X9_down, y9_down)  
BalancedRandomForestClassifier(...)

brfc_pred9 = clf9.predict(X9_test)
brfc_cr9 = classification_report(y9_test, brfc_pred9, digits=10)
print(brfc_cr9)

145988 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 145988 events and 257 original time points ...
42 bad epochs dropped
              precision    recall  f1-score   support

           0  0.9999705848 0.9345447548 0.9661513102     36376
           1  0.0441589723 0.9909909910 0.0845503459       111

    accuracy                      0.9347164744     36487
   macro avg  0.5220647785 0.9627678729 0.5253508280     36487
weighted avg  0.9970628344 0.9347164744 0.9634693219     36487



In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
X_2d, y, stratify=y)

# concatenate our training data back together
X_stack = np.column_stack((X[train], y[train]))

# separate minority and majority classes
a = y[train] == 1
seizures = X_stack[a,:]
nonseizures = X_stack[~a,:]
nonseizures_downsampled = resample(nonseizures,
                                replace = False, # sample without replacement
                                n_samples = int(10*len(seizures)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = np.vstack((seizures, nonseizures_downsampled))
shuff_down = shuffle(downsampled)

y_down = shuff_down[:,-1]
X_down = np.delete(shuff_down, -1, axis=1)

clf9 = BalancedRandomForestClassifier(n_estimators=300, max_depth=3, random_state=0)
clf9.fit(X9_down, y9_down)  
BalancedRandomForestClassifier(...)

brfc_pred9 = clf9.predict(X9_test)
brfc_cr9 = classification_report(y9_test, brfc_pred9, digits=10)
print(brfc_cr9)

In [ ]:
def check_extension(filename:str, ext:str='.png') -> str:
    """
    Verify if a filename ends with a desired extension, coerces if not
    
    Args:
        filename (str): input filename [e.g. 'plot.png']
        ext (str): desired extension [default: '.png']
        
    Returns:
        str: output filename, original if valid ext
        
    """
    # Add '.' to extension if not input
    
    if not filename.endswith(ext): 
        new_filename = filename + ext
        print(f"Filename changed from '{filename}' to '{new_filename}'")
        return new_filename
    else:
        return filename


def plot_binary_ROC(clf, X, y, scaler=None, cv_fold:int=3, pos_label_:int=None, dpi_:int=150, png_filename:str=None, verbose:bool=False):
    """
    Plot a ROC curve for a binary classifier with n='cv-fold' cross-validations
    
    Args:
        clf:          untrained classifier object (e.g. rf_clf = RandomForestClassifer())
        X:            training + testing data
        y:            targets (numeric/integers)
        scaler:       scaler to rescale data, e.g. StandardScaler() [default: None]
        cv_fold:      cross-validations to run [default: 3]
        dpi_:         dpi resolution for plot (x,y) [default: 150] 
        pos_label_:   if targets are not binary (0, 1) then indicate integer for "positive" [default: None]
        png_filename: if you wish to save to png, give filename [default: None]
    
    Returns:
        clf: trained scikit learn classifer object
    
    LEGEND:
        - AUC: area under ROC curve
        - TPR: true positive rate
        - FPR: false positive rate
        - AP: average precision
        - Max F1: maximum F1 score
    """
    
    """ Setup """
    
    from sklearn.model_selection import train_test_split, StratifiedKFold  # sklearn > v0.18

    # rescale data
    if scaler is not None:
        X = scaler.fit_transform(X)
        if verbose:
            print(f"Data (X) rescaled using {scaler}.")

    # Set up mean true and false positive rates
    # -- mean_tpr begins as 0, and we'll update it as we fit the classifiers
    # -- mean_fpr is our x-axis so we pre-calculate it from 0 to 100 (%)
    mean_tpr, mean_fpr = 0, np.linspace(0, 1, 101) # np.linspace(start, stop, datapoints)
    mean_prec, mean_rec = 0, np.linspace(0, 1, 101) # np.linspace(start, stop, datapoints)

    # Set the cross-validation fold
    if cv_fold > 1:
        skf = StratifiedKFold(cv_fold)
    else:
        print(f"ERROR: cv_fold set as {cv_fold}, but must be > 1.")
        return clf

    """ Plotting """
    
    # Create new plot
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, dpi=dpi_)
    
    # Set the matplotlib colorwheel as a cycle
    colors = cycle(list(cnames.keys()))
    
    # Loop over each split in the data and plot the ROC for each cv_fold
    # -- skf.split(X,y) returns n=cv_fold splits of the data
    # -- zip(skf..., colors) will return a list of tuples with the split data (itself a tuple) and a color
    # -- each loop will therefore get its own split of the data and a color for plotting
    # -- idx, val in enumerate(...) returns the index (idx) of our loop (0, 1, 2... cv_fold-1)
    #        and the actual value i.e. the split data and color
    for idx, val in enumerate(zip(skf.split(X, y), colors)):
        
        # retrieve training and test data and color/labels
        (train, test), color = val
        # concatenate our training data back together
        X_stack = np.column_stack((X[train], y[train]))
        
        # separate minority and majority classes
        a = y[train] == 1
        seizures = X_stack[a,:]
        nonseizures = X_stack[~a,:]
        nonseizures_downsampled = resample(nonseizures,
                                        replace = False, # sample without replacement
                                        n_samples = int(10*len(seizures)), # match minority n
                                        random_state = 27) # reproducible results

        # combine minority and downsampled majority
        downsampled = np.vstack((seizures, nonseizures_downsampled))
        shuff_down = shuffle(downsampled)

        y_down = shuff_down[:,-1]
        X_down = np.delete(shuff_down, -1, axis=1)

        # train and retrieve probabilities of class per feature for the test data
        # probas_ will have a shape of n_instances by 2 classes, with the sum of each row = 1
        try:
            probas_ = clf.fit(X_down, y_down).predict_proba(X[test])
        except:
            print("Check if classifer allows for probability calculation.")
            break

        # get false and true positive rates (and thresholds, which is not used but shown here for fyi)
        try:
            fpr, tpr, _ = roc_curve(y[test], probas_[:,1], pos_label=pos_label_)
            precision, recall, _ = precision_recall_curve(y[test], probas_[:,1], pos_label=pos_label_)
            f1_score = 2*(precision*recall)/(precision+recall)
            avg_precision = average_precision_score(y[test], probas_[:,1])
        except:
            print(f"Warning: target data not binary. Maximum target value of '{np.max(y)}' used as positive.")
            print("You can use 'pos_label_' to indicate your own.")
            fpr, tpr, _ = roc_curve(y[test], probas_[:, 1], pos_label=np.max(y))
            precision, recall, _ = precision_recall_curve(y[test], probas_[:,1], pos_label=np.max(y))
            f1_score = 2*(precision*recall)/(precision+recall)
            avg_precision = average_precision_score(y[test], probas_[:,1])

        # get area under the curve
        roc_auc = auc(fpr, tpr)
        
        # recalculate mean_tpr curve w/ linear interpolation with mean_fpr, fpr, and tpr
        mean_tpr += np.interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0

        # recalculate mean_prec curve w/ linear interpolation
        mean_prec += np.interp(mean_fpr, recall, precision)
        
        # PLOT current ROC curve
        ax1.plot(fpr, tpr, lw=1, color=color, label=f'AUC = {roc_auc:.3f}')
        
        # PLOT precision-recall curve with max f1_score
        max_f1 = np.max(f1_score)
        max_f1_recall = recall[np.argmax(f1_score)]
        ax2.plot(recall, precision, lw=1, color=color, label=f'AP = {avg_precision:.3f}, Max F1 = {max_f1:.2f}')
        ax2.plot(max_f1_recall, max_f1, 'o', color=color, label=None)

    # ROC - PLOT avg roc of all curves
    mean_tpr /= skf.get_n_splits(X, y) # normalize by dividing by number of splits
    mean_tpr[-1] = 1.0 # constrain last index to 1
    mean_auc = auc(mean_fpr, mean_tpr)
    ax1.plot(mean_fpr, mean_tpr, color='k', linestyle='--', label=f'$\mu$(AUC) = {mean_auc:.3f}', lw=0.75)

    # PLOT chance (tpr = fpr)
    ax1.plot([0, 1], [0, 1], linestyle='--', lw=0.5, color='k')

    # ROC - Axes and labels
    ax1.set_xlim([-0.05, 1.05])
    ax1.set_ylim([-0.05, 1.05])
    ax1.set_xlabel('False Positive Rate')
    ax1.set_ylabel('True Positive Rate (Recall)')
    ax1.set_title(f'ROC curve ({cv_fold}-fold CV)')
    ax1.legend(loc="lower right", prop={'size':6})

    # PR - Axes and labels
    ax2.set_xlim([-0.05, 1.05])
    ax2.set_ylim([-0.05, 1.05])
    ax2.set_xlabel('True Positive Rate (Recall)')
    ax2.set_ylabel('Precision')
    ax2.set_title(f'Precision-Recall curve ({cv_fold}-fold CV)')
    ax2.legend(loc="lower right", prop={'size':6})
    
    # set tight layout to fix spacing
    plt.tight_layout()

    # output
    if not png_filename:
        plt.show()
    else:
        png_filename = check_extension(png_filename)
        plt.savefig(png_filename)
        if verbose:
            print(f"Figure saved as '{png_filename}'")

    return clf


# ROC on a Balanced Random Forest Classifier trained on 3 folds 
balrf_clf = BalancedRandomForestClassifier(n_estimators=300, max_depth=3, random_state=0)
# clf.fit(X_down, y_down)  
# BalancedRandomForestClassifier(...)

# brfc_pred9 = clf9.predict(X9_test)
# brfc_cr9 = classification_report(y9_test, brfc_pred9, digits=10)
# print(brfc_cr9)
balrf_clf = plot_binary_ROC(balrf_clf, X_2D, y, cv_fold=5, dpi_=150, png_filename='roc_test.png')

In [9]:
X9_2d = X9.reshape(len(X9), -1)

In [10]:
from sklearn.model_selection import train_test_split
X9_train, X9_test, y9_train, y9_test = train_test_split(
X9_2d, y9, stratify=y9)

In [11]:
y9_test.sum() / len(y9_test), len(y9_test), y9_test.sum()

(0.0020427006653006502, 158124, 323)

In [12]:
unique_elements, counts_elements = np.unique(y9_train, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[     0      1]
 [473402    969]]


In [13]:
# concatenate our training data back together
X9_stack = np.column_stack((X9_train, y9_train))

# separate minority and majority classes
a = y9_train == 1
seizures = X9_stack[a,:]
nonseizures = X9_stack[~a,:]
nonseizures_downsampled = resample(nonseizures,
                                replace = False, # sample without replacement
                                n_samples = int(10*len(seizures)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled9 = np.vstack((seizures, nonseizures_downsampled))
shuff_down9 = shuffle(downsampled9)

y9_down = shuff_down9[:,-1]
X9_down = np.delete(shuff_down9, -1, axis=1)
# checking counts
len(seizures), len(nonseizures_downsampled)

(969, 9690)

In [14]:
clf9 = BalancedRandomForestClassifier(n_estimators=300, max_depth=3, random_state=0)
clf9.fit(X9_down, y9_down)  
BalancedRandomForestClassifier(...)
print(clf9.feature_importances_) 

[0.00000000e+00 8.40397392e-06 0.00000000e+00 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [15]:
brfc_pred9 = clf9.predict(X9_test)
brfc_cr9 = classification_report(y9_test, brfc_pred9, digits=10)
print(brfc_cr9)

              precision    recall  f1-score   support

           0  0.9998235423 0.8976622455 0.9459926940    157801
           1  0.0181188059 0.9226006192 0.0355396541       323

    accuracy                      0.8977131871    158124
   macro avg  0.5089711741 0.9101314323 0.4907661740    158124
weighted avg  0.9978182134 0.8977131871 0.9441329109    158124



In [7]:

event_id = dict(Seizure=1, Nonseizure=0)
events_from_annot, event_dict = mne.events_from_annotations(raws, event_id=event_id, chunk_duration=1)

tmin, tmax = -.5, .5
epochs = mne.Epochs(raws, events_from_annot, event_id, tmin, tmax, proj=True,
                    decim=2, baseline=None, preload=True)

y9 = epochs.events[:, 2]

from mne.time_frequency import psd_welch
psds9, freqs9 = psd_welch(epochs, fmin=1, fmax=40, n_fft=128, n_overlap=.5, n_per_seg=128, proj=False, n_jobs=-1, reject_by_annotation=True, average='mean', verbose=None)
psds9 /= np.sum(psds9, axis=-1, keepdims=True)
psds_rs9 = psds9.reshape(len(psds9), -1)

from sklearn.model_selection import train_test_split
X9p_train, X9p_test, y9p_train, y9p_test = train_test_split(
psds_rs9, y9, stratify=y9)

# concatenate our training data back together
X9p_stack = np.column_stack((X9p_train, y9p_train))

# separate minority and majority classes
a = y9p_train == 1
seizures = X9p_stack[a,:]
nonseizures = X9p_stack[~a,:]
nonseizures_downsampled = resample(nonseizures,
                                replace = False, # sample without replacement
                                n_samples = int(10*len(seizures)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled9p = np.vstack((seizures, nonseizures_downsampled))
shuff_down9p = shuffle(downsampled9p)

y9p_down = shuff_down9p[:,-1]
X9p_down = np.delete(shuff_down9p, -1, axis=1)

clf9p = BalancedRandomForestClassifier(n_estimators=300, max_depth=3, random_state=0)
clf9p.fit(X9p_down, y9p_down)  
BalancedRandomForestClassifier(...)
print(clf9p.feature_importances_) 

brfc_pred9p = clf9p.predict(X9p_test)
brfc_cr9p = classification_report(y9p_test, brfc_pred9p, digits=10)
print(brfc_cr9p)

Used Annotations descriptions: ['Nonseizure', 'Seizure']
863699 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 863699 events and 257 original time points ...
154 bad epochs dropped
Effective window size : 1.000 (s)


ValueError: number sections must be larger than 0.

In [18]:
from sklearn.model_selection import train_test_split
X9p_train, X9p_test, y9p_train, y9p_test = train_test_split(
psds_rs9, y9, stratify=y9)

In [19]:
# concatenate our training data back together
X9p_stack = np.column_stack((X9p_train, y9p_train))

# separate minority and majority classes
a = y9p_train == 1
seizures = X9p_stack[a,:]
nonseizures = X9p_stack[~a,:]
nonseizures_downsampled = resample(nonseizures,
                                replace = False, # sample without replacement
                                n_samples = int(10*len(seizures)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled9p = np.vstack((seizures, nonseizures_downsampled))
shuff_down9p = shuffle(downsampled9p)

y9p_down = shuff_down9p[:,-1]
X9p_down = np.delete(shuff_down9p, -1, axis=1)
# checking counts
len(seizures), len(nonseizures_downsampled)

(667, 6670)

In [20]:
clf9p = BalancedRandomForestClassifier(n_estimators=300, max_depth=3, random_state=0)
clf9p.fit(X9p_down, y9p_down)  
BalancedRandomForestClassifier(...)
print(clf9p.feature_importances_) 

[0.00000000e+00 7.89948603e-04 4.82446619e-04 1.99476933e-04
 0.00000000e+00 5.19040498e-04 1.77625128e-04 8.12890825e-05
 2.75742008e-04 0.00000000e+00 1.02005027e-03 1.24780278e-03
 3.40291096e-04 2.71616226e-04 0.00000000e+00 4.21382947e-03
 3.62329198e-04 1.51791135e-03 3.44138386e-04 0.00000000e+00
 5.34875225e-04 8.28170160e-05 0.00000000e+00 7.78278718e-05
 8.32870702e-05 0.00000000e+00 4.00527944e-04 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.90539563e-04
 0.00000000e+00 3.24532356e-04 0.00000000e+00 1.61940651e-04
 0.00000000e+00 1.18670447e-04 0.00000000e+00 1.76626116e-04
 2.02590290e-03 0.00000000e+00 3.47246147e-03 1.89282381e-04
 1.56479922e-04 2.10874473e-04 6.75066494e-05 6.38067349e-04
 0.00000000e+00 7.47939432e-04 2.00679808e-04 0.00000000e+00
 1.90703065e-04 2.30735571e-04 8.58970461e-05 0.00000000e+00
 2.79732278e-04 1.10960729e-04 4.12420159e-04 0.00000000e+00
 2.12478609e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.000000

In [21]:
brfc_pred9p = clf9p.predict(X9p_test)
brfc_cr9p = classification_report(y9p_test, brfc_pred9p, digits=10)
print(brfc_cr9p)

              precision    recall  f1-score   support

           0  0.9995612445 0.8224970960 0.9024257925    127412
           1  0.0077655421 0.7937219731 0.0153806048       223

    accuracy                      0.8224468210    127635
   macro avg  0.5036633933 0.8081095346 0.4589031986    127635
weighted avg  0.9978284092 0.8224468210 0.9008759740    127635



In [60]:

event_id = dict(Seizure=1, Nonseizure=0)
events_from_annot, event_dict = mne.events_from_annotations(raws, event_id=event_id, chunk_duration=2)

Used Annotations descriptions: ['Nonseizure', 'Seizure']


In [61]:
events_from_annot, np.count_nonzero(events_from_annot, axis=0)

(array([[16803072,        0,        0],
        [16803584,        0,        0],
        [16804096,        0,        0],
        ...,
        [62548736,        0,        0],
        [62549248,        0,        0],
        [62549760,        0,        0]]), array([14680,     0,   137]))

In [74]:
tmin, tmax = -.5, .5
epochs = mne.Epochs(raws, events_from_annot, event_id, tmin, tmax, proj=True,
                    decim=2, baseline=None, preload=True)



# get MEG and EEG data

picks = mne.pick_types(epochs.info, meg=False, eeg=True)
X9p2 = epochs.get_data()[:, picks]
y9p2 = epochs.events[:, 2]
X9p2.shape, y9p2.shape

X9p2_2d = X9p2.reshape(len(X9p2), -1)

14680 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 14680 events and 257 original time points ...
3 bad epochs dropped


In [75]:

from mne.time_frequency import psd_array_welch
from mne.time_frequency import psd_welch
psds9_2, freqs9_2 = psd_welch(epochs, fmin=1, fmax=40, n_fft=128, n_overlap=.5, n_per_seg=128, proj=False, n_jobs=1, reject_by_annotation=True, average='mean', verbose=None)
psds9_2 /= np.sum(psds9_2, axis=-1, keepdims=True)
psds_rs9_2 = psds9_2.reshape(len(psds9_2), -1)

Effective window size : 1.000 (s)


In [78]:
y9p2.shape, psds_rs9_2.shape

((14677,), (14677, 920))

In [80]:
from sklearn.model_selection import train_test_split
X9p2_train, X9p2_test, y9p2_train, y9p2_test = train_test_split(
psds_rs9_2, y9p2, stratify=y9p2)

In [81]:
X9p2_stack = np.column_stack((X9p2_train, y9p2_train))

# separate minority and majority classes
a = y9p2_train == 1
seizures = X9p2_stack[a,:]
nonseizures = X9p2_stack[~a,:]
nonseizures_downsampled = resample(nonseizures,
                                replace = False, # sample without replacement
                                n_samples = int(10*len(seizures)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled9p2 = np.vstack((seizures, nonseizures_downsampled))
shuff_down9p2 = shuffle(downsampled9p2)

y9p2_down = shuff_down9p2[:,-1]
X9p2_down = np.delete(shuff_down9p2, -1, axis=1)
# checking counts
len(seizures), len(nonseizures_downsampled)

(103, 1030)

In [83]:
clf9p2 = BalancedRandomForestClassifier(n_estimators=300, max_depth=3, random_state=0)
clf9p2.fit(X9p2_down, y9p2_down)  
BalancedRandomForestClassifier(...)
print(clf9p2.feature_importances_) 

[7.39633825e-04 0.00000000e+00 2.15108215e-03 0.00000000e+00
 4.36042717e-04 5.16018003e-04 1.91852962e-04 1.29709660e-03
 2.46461223e-04 3.34212988e-05 1.14780744e-03 9.85944601e-04
 7.49231430e-04 1.30486872e-03 1.81858133e-03 3.01110215e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.11571732e-04
 1.54644436e-03 4.79801631e-04 5.31983235e-04 0.00000000e+00
 1.51296502e-04 9.67492114e-04 1.64967318e-03 1.34988432e-03
 1.62075651e-04 1.04782126e-04 1.74285401e-03 2.41147422e-04
 6.40783643e-04 1.90199844e-03 0.00000000e+00 0.00000000e+00
 6.98563936e-04 4.38900141e-05 3.38926004e-04 4.05563390e-04
 5.01881407e-03 1.52590951e-04 0.00000000e+00 2.81196438e-04
 0.00000000e+00 1.24464675e-03 8.56368222e-04 1.88033226e-03
 6.55848274e-03 3.04553605e-03 4.36295179e-03 2.43201195e-03
 3.02865441e-03 1.20069715e-03 2.28559864e-04 6.10779742e-04
 6.01092665e-05 0.00000000e+00 3.18786318e-04 2.01109078e-03
 4.04602449e-04 7.99684102e-04 6.39053626e-04 1.41451036e-03
 0.00000000e+00 2.377090

In [84]:
brfc_pred9p2 = clf9p2.predict(X9p2_test)
brfc_cr9p2 = classification_report(y9p2_test, brfc_pred9p2, digits=10)
print(brfc_cr9p2)

              precision    recall  f1-score   support

           0  0.9984251969 0.8718371837 0.9308471590      3636
           1  0.0585858586 0.8529411765 0.1096408318        34

    accuracy                      0.8716621253      3670
   macro avg  0.5285055277 0.8623891801 0.5202439954      3670
weighted avg  0.9897182384 0.8716621253 0.9232392530      3670



In [12]:
fif1_lst = [rec_name + '_raw.fif' for rec_name in rec_wos if rec_name[0:5] == 'chb01']

In [13]:


data_path = 'fif/'

raws = mne.concatenate_raws([mne.io.read_raw_fif('fif/' + f, preload=False) for f in fif1_lst])
info = raws.info
channels = raws.ch_names

Opening raw data file fif/chb01_01_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif/chb01_02_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif/chb01_03_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif/chb01_04_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif/chb01_05_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif/chb01_06_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif/chb01_07_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif/chb01_08_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fif/chb01_09_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening ra

In [14]:

event_id = dict(Seizure=1, Nonseizure=0)
events_from_annot, event_dict = mne.events_from_annotations(raws, event_id=event_id, chunk_duration=1)

Used Annotations descriptions: ['Nonseizure', 'Seizure']


In [15]:
events_from_annot, np.count_nonzero(events_from_annot, axis=0)

(array([[       0,        0,        0],
        [     256,        0,        0],
        [     512,        0,        0],
        ...,
        [37372160,        0,        0],
        [37372416,        0,        0],
        [37372672,        0,        0]]), array([145987,      0,    442]))

In [16]:
tmin, tmax = -0.5, 0.5
epochs = mne.Epochs(raws, events_from_annot, event_id, tmin, tmax, proj=True,
                    decim=2, baseline=None, preload=True)

labels = epochs.events[:, -1]

# get EEG data
# fif_epochs = epochs.copy().pick_types(meg=False, eeg=True)
# fif_data = fif_epochs.get_data().reshape(len(labels), -1)
picks = mne.pick_types(epochs.info, meg=False, eeg=True)
X1 = epochs.get_data()[:, picks]
y1 = epochs.events[:, 2]
X1.shape, y1.shape



145988 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 145988 events and 257 original time points ...
42 bad epochs dropped


((145946, 23, 129), (145946,))

In [18]:
X1_2d = X1.reshape(len(X1), -1)

In [19]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y1_train, y1_test = train_test_split(
X1_2d, y1, stratify=y1)



In [20]:
y1_test.sum() / len(y1_test), len(y1_test), y1_test.sum()

(0.0030421794063639103, 36487, 111)

In [21]:
unique_elements, counts_elements = np.unique(y1_train, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[     0      1]
 [109128    331]]


In [22]:
# concatenate our training data back together
X1_stack = np.column_stack((X1_train, y1_train))

# separate minority and majority classes
a = y1_train == 1
seizures = X1_stack[a,:]
nonseizures = X1_stack[~a,:]
nonseizures_downsampled = resample(nonseizures,
                                replace = False, # sample without replacement
                                n_samples = int(5*len(seizures)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = np.vstack((seizures, nonseizures_downsampled))
shuff_down = shuffle(downsampled)
# checking counts
len(seizures), len(nonseizures_downsampled)

(331, 1655)

In [95]:
shuff_down2.shape, shuff_down2[:,-1], shuff_down2.sum()

((1986, 2968), array([0., 0., 1., ..., 0., 0., 1.]), 331.16931014637856)

In [23]:

X1_down = np.delete(shuff_down, -1, axis=1)

y1_down = shuff_down[:,-1]

In [25]:
clf1 = BalancedRandomForestClassifier(n_estimators=300, max_depth=3, random_state=0)
clf1.fit(X1_down, y1_down)  
BalancedRandomForestClassifier(...)
print(clf1.feature_importances_) 

[0.00000000e+00 8.21341795e-05 0.00000000e+00 ... 0.00000000e+00
 7.61771520e-06 0.00000000e+00]


In [27]:
brfc_pred1 = clf1.predict(X1_test)
brfc_cr1 = classification_report(y1_test, brfc_pred1, digits=10)
print(brfc_cr1)

              precision    recall  f1-score   support

           0  0.9997963399 0.9446888058 0.9714616874     36376
           1  0.0491493384 0.9369369369 0.0933991917       111

    accuracy                      0.9446652232     36487
   macro avg  0.5244728392 0.9408128714 0.5324304396     36487
weighted avg  0.9969043012 0.9446652232 0.9687904638     36487



In [28]:
# mne.Epochs.next()
from mne.time_frequency import psd_array_welch
from mne.time_frequency import psd_welch
psds, freqs = psd_welch(epochs, fmin=1, fmax=40, n_fft=128, n_overlap=.5, n_per_seg=128, proj=False, n_jobs=1, reject_by_annotation=True, average='mean', verbose=None)
psds /= np.sum(psds, axis=-1, keepdims=True)
psds_rs = psds.reshape(len(psds), -1)

Effective window size : 1.000 (s)


In [30]:
from sklearn.model_selection import train_test_split
X1p_train, X1p_test, y1p_train, y1p_test = train_test_split(
psds_rs, y1, stratify=y1)

In [37]:
# concatenate our training data back together
X1p_stack = np.column_stack((X1p_train, y1p_train))


# separate minority and majority classes
a = y1p_train == 1
seizures = X1p_stack[a,:]
nonseizures = X1p_stack[~a,:]
nonseizures_downsampled = resample(nonseizures,
                                replace = False, # sample without replacement
                                n_samples = int(5*len(seizures)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled1p = np.vstack((seizures, nonseizures_downsampled))
shuff_down1p = shuffle(downsampled1p)

X1p_down = np.delete(shuff_down1p, -1, axis=1)

y1p_down = shuff_down1p[:,-1]
# checking counts
len(seizures), len(nonseizures_downsampled)

(331, 1655)

In [38]:
clf1p = BalancedRandomForestClassifier(n_estimators=300, max_depth=3, random_state=0)
clf1p.fit(X1p_down, y1p_down)  
BalancedRandomForestClassifier(...)
print(clf1p.feature_importances_) 

[1.11609729e-04 4.11033303e-04 0.00000000e+00 4.53931863e-05
 0.00000000e+00 0.00000000e+00 2.37316974e-05 0.00000000e+00
 0.00000000e+00 1.74629631e-04 7.62757512e-04 1.09223057e-04
 3.29573102e-04 1.12420832e-04 0.00000000e+00 9.45634674e-04
 0.00000000e+00 9.74902344e-05 2.03392685e-04 0.00000000e+00
 1.71539242e-04 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 5.59327711e-05
 0.00000000e+00 8.67565346e-05 0.00000000e+00 2.67496767e-03
 7.15445433e-05 0.00000000e+00 0.00000000e+00 2.04435905e-04
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.25022790e-04 0.00000000e+00 2.53492621e-04 0.00000000e+00
 3.90592528e-05 0.00000000e+00 0.00000000e+00 1.65962002e-04
 0.00000000e+00 2.21071698e-04 1.02575686e-04 0.00000000e+00
 0.00000000e+00 1.02245922e-04 8.77644070e-05 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.21065797e-04
 4.97709117e-05 0.000000

In [39]:
brfc_pred1p = clf1p.predict(X1p_test)
brfc_cr1p = classification_report(y1p_test, brfc_pred1p, digits=10)
print(brfc_cr1p)

              precision    recall  f1-score   support

           0  0.9998783714 0.9039751484 0.9495112831     36376
           1  0.0297222222 0.9639639640 0.0576663972       111

    accuracy                      0.9041576452     36487
   macro avg  0.5148002968 0.9339695562 0.5035888401     36487
weighted avg  0.9969269823 0.9041576452 0.9467981309     36487



In [32]:
brfc_pred3 = clf2.predict(X3_test)
brfc_cr3 = classification_report(y3_test, brfc_pred3, digits=10)
print(brfc_cr3)

              precision    recall  f1-score   support

           0  0.9998952587 0.9147210067 0.9554135878     31309
           1  0.0147601476 0.9302325581 0.0290592081        43

    accuracy                      0.9147422812     31352
   macro avg  0.5073277032 0.9224767824 0.4922363979     31352
weighted avg  0.9985441229 0.9147422812 0.9541430711     31352



In [34]:
brfc_pred4 = clf.predict(X2_test)
brfc_cr4 = classification_report(y2_test, brfc_pred4, digits=10)
print(brfc_cr4)

              precision    recall  f1-score   support

           0  0.9985684790 0.9971684627 0.9978679798     36376
           1  0.3641975309 0.5315315315 0.4322344322       111

    accuracy                      0.9957519116     36487
   macro avg  0.6813830049 0.7643499971 0.7150512060     36487
weighted avg  0.9966386088 0.9957519116 0.9961472211     36487



In [5]:
fif2_lst = [rec_name + '_raw.fif' for rec_name in rec_wos if rec_name[0:5] == 'chb02']

In [6]:


raws = mne.concatenate_raws([mne.io.read_raw_fif('fifch2/' + f, preload=False) for f in fif2_lst])
info = raws.info
channels = raws.ch_names

Opening raw data file fifch2/chb02_01_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fifch2/chb02_02_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fifch2/chb02_03_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fifch2/chb02_04_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fifch2/chb02_05_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fifch2/chb02_06_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fifch2/chb02_07_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fifch2/chb02_08_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599.996 secs
Ready.
Opening raw data file fifch2/chb02_09_raw.fif...
    Range : 0 ... 921599 =      0.000 ...  3599

In [7]:

event_id = dict(Seizure=1, Nonseizure=0)
events_from_annot, event_dict = mne.events_from_annotations(raws, event_id=event_id, chunk_duration=1)

Used Annotations descriptions: ['Nonseizure', 'Seizure']


In [8]:
events_from_annot, np.count_nonzero(events_from_annot, axis=0)

(array([[       0,        0,        0],
        [     256,        0,        0],
        [     512,        0,        0],
        ...,
        [32500736,        0,        0],
        [32500992,        0,        0],
        [32501248,        0,        0]]), array([125442,      0,    172]))

In [9]:
tmin, tmax = -0.5, 0.5
epochs = mne.Epochs(raws, events_from_annot, event_id, tmin, tmax, proj=True,
                    decim=2, baseline=None, preload=True)

labels = epochs.events[:, -1]

# get MEG and EEG data
fif_epochs = epochs.copy().pick_types(meg=False, eeg=True)
fif_data = fif_epochs.get_data().reshape(len(labels), -1)
picks = mne.pick_types(epochs.info, meg=False, eeg=True)
X3 = epochs.get_data()[:, picks]
y3 = epochs.events[:, 2]
X3.shape, y3.shape

125443 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 125443 events and 257 original time points ...
36 bad epochs dropped


((125407, 23, 129), (125407,))

In [10]:
X3_2d = X3.reshape(len(X3), -1)

In [11]:
from sklearn.model_selection import train_test_split
X3_train, X3_test, y3_train, y3_test = train_test_split(
X3_2d, y3, stratify=y3)

In [12]:
y3_test.sum() / len(y3_test), len(y3_test), y3_test.sum()

(0.0013715233477928042, 31352, 43)

In [13]:
unique_elements, counts_elements = np.unique(y3_train, return_counts=True)
print("Frequency of unique values of the said array:")
print(np.asarray((unique_elements, counts_elements)))

Frequency of unique values of the said array:
[[    0     1]
 [93926   129]]


In [15]:
# concatenate our training data back together
X3_stack = np.column_stack((X3_train, y3_train))

# separate minority and majority classes
a = y3_train == 1
seizures = X3_stack[a,:]
nonseizures = X3_stack[~a,:]
nonseizures_downsampled = resample(nonseizures,
                                replace = False, # sample without replacement
                                n_samples = int(10*len(seizures)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled3 = np.vstack((seizures, nonseizures_downsampled))
shuff_down = shuffle(downsampled3)

y_down = shuff_down[:,-1]
X_down = np.delete(shuff_down, -1, axis=1)
# checking counts
len(seizures), len(nonseizures_downsampled)

(129, 1290)

In [17]:
clf5 = BalancedRandomForestClassifier(n_estimators=300, max_depth=3, random_state=0)
clf5.fit(X_down, y_down)  
BalancedRandomForestClassifier(...)
print(clf5.feature_importances_) 

[9.64643465e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
 0.00000000e+00 0.00000000e+00]


In [18]:
brfc_pred5 = clf5.predict(X3_test)
brfc_cr5 = classification_report(y3_test, brfc_pred5, digits=10)
print(brfc_cr5)

              precision    recall  f1-score   support

           0  0.9998315080 0.9476508352 0.9730421094     31309
           1  0.0226595110 0.8837209302 0.0441860465        43

    accuracy                      0.9475631539     31352
   macro avg  0.5112455095 0.9156858827 0.5086140780     31352
weighted avg  0.9984912938 0.9475631539 0.9717681616     31352



In [53]:
brfc_pred5 = clf5.predict(X2_test)
brfc_cr5 = classification_report(y2_test, brfc_pred5, digits=10)
print(brfc_cr5)

              precision    recall  f1-score   support

           0  0.9995683329 0.9548603475 0.9767029877     36376
           1  0.0552359033 0.8648648649 0.1038399135       111

    accuracy                      0.9545865651     36487
   macro avg  0.5274021181 0.9098626062 0.5402714506     36487
weighted avg  0.9966955042 0.9545865651 0.9740475816     36487



In [54]:
brfc_pred5 = clf5.predict(X_test)
brfc_cr5 = classification_report(y_test, brfc_pred5, digits=10)
print(brfc_cr5)

              precision    recall  f1-score   support

           0  0.9999565718 0.7710119207 0.8706856619     59728
           1  0.0048748545 0.9710144928 0.0097010063        69

    accuracy                      0.7712427045     59797
   macro avg  0.5024157131 0.8710132067 0.4401933341     59797
weighted avg  0.9988083430 0.7712427045 0.8696921682     59797



In [55]:
brfc_pred = clf.predict(X3_test)
brfc_cr = classification_report(y3_test, brfc_pred, digits=10)
print(brfc_cr)

              precision    recall  f1-score   support

           0  0.9993262320 0.9948257689 0.9970709221     31309
           1  0.1195652174 0.5116279070 0.1938325991        43

    accuracy                      0.9941630518     31352
   macro avg  0.5594457247 0.7532268380 0.5954517606     31352
weighted avg  0.9981196193 0.9941630518 0.9959692620     31352



In [56]:
brfc_pred2 = clf2.predict(X3_test)
brfc_cr2 = classification_report(y3_test, brfc_pred2, digits=10)
print(brfc_cr2)

              precision    recall  f1-score   support

           0  1.0000000000 0.9207576096 0.9587442007     31309
           1  0.0170364501 1.0000000000 0.0335021426        43

    accuracy                      0.9208662924     31352
   macro avg  0.5085182250 0.9603788048 0.4961231717     31352
weighted avg  0.9986518425 0.9208662924 0.9574752097     31352



In [11]:
tmin, tmax = -0.5, 0.5
epochs = mne.Epochs(raws, events_from_annot, event_id, tmin, tmax, proj=True,
                    decim=2, baseline=None, preload=True)

labels = epochs.events[:, -1]

# get MEG and EEG data
fif_epochs = epochs.copy().pick_types(meg=False, eeg=True)
fif_data = fif_epochs.get_data().reshape(len(labels), -1)
picks = mne.pick_types(epochs.info, meg=False, eeg=True)
X = epochs.get_data()[:, picks]
y = epochs.events[:, 2]
X.shape, y.shape


145988 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 145988 events and 257 original time points ...
42 bad epochs dropped


((145946, 23, 129), (145946,))

In [10]:
tmin, tmax = -0.5, 0.5
epochs = mne.Epochs(raws, events_from_annot, event_id, tmin, tmax, proj=True,
                    decim=2, baseline=None, preload=True)



    



125443 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
Loading data for 125443 events and 257 original time points ...
36 bad epochs dropped


In [17]:

y = epochs.events[:, 2]
y.sum()

172

In [9]:
picks = mne.pick_types(epochs.info, meg=False, eeg=True)
X = epochs.get_data()[:, picks]

In [36]:
X.shape, y.shape, X3_2d = X3.reshape(len(X3), -1), 

NameError: name 'X3' is not defined

In [15]:
# mne.Epochs.next()
from mne.time_frequency import psd_array_welch
from mne.time_frequency import psd_welch
psds, freqs = psd_welch(epochs, fmin=1, fmax=40, n_fft=128, n_overlap=.5, n_per_seg=128, proj=False, n_jobs=1, reject_by_annotation=True, average='mean', verbose=None)
psds /= np.sum(psds, axis=-1, keepdims=True)
psds_rs = psds.reshape(len(psds), -1)

Effective window size : 1.000 (s)


In [31]:
psds /= np.sum(psds, axis=-1, keepdims=True)

In [32]:
psds_rs = psds.reshape(len(psds), -1)

In [33]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# def create_fft(fn):
#     sample_rate, X = scipy.io.wavfile.read(fn)
#     fft_features = abs(scipy.fft(X)[:1000])
#     base_fn, ext = os.path.splitext(fn)
#     data_fn = base_fn + ".fft"
#     np.save(data_fn, fft_features)

In [17]:
type(psds), type(freqs), psds.shape, freqs.shape, psds_rs.shape, y.shape

# if input is of type Epochs, then psds will be of shape (n_epochs, n_channels, n_freqs)

(numpy.ndarray,
 numpy.ndarray,
 (145946, 23, 40),
 (40,),
 (145946, 920),
 (145946,))

In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
psds_rs, y, stratify=y)

In [35]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, y_train.sum(), y_test.sum()

((94055, 920), (31352, 920), (94055,), (31352,), 129, 43)

In [36]:
# concatenate our training data back together
X_stack = np.column_stack((X_train, y_train))


# separate minority and majority classes
a = y_train == 1
seizures = X_stack[a,:]
nonseizures = X_stack[~a,:]
nonseizures_downsampled = resample(nonseizures,
                                replace = False, # sample without replacement
                                n_samples = int(5*len(seizures)), # match minority n
                                random_state = 27) # reproducible results

# combine minority and downsampled majority
downsampled = np.vstack((seizures, nonseizures_downsampled))

# checking counts
len(seizures), len(nonseizures_downsampled)

(129, 645)

In [22]:
X_stack[:,-1], downsampled[:,-1]

(array([0., 0., 0., ..., 0., 0., 0.]),
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [37]:
shuff_down = shuffle(downsampled)
shuff_down[:,-1]

array([0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 1., 0., 0., 0.

In [38]:


X7_down = np.delete(shuff_down, -1, axis=1)

y7_down = shuff_down[:,-1]

In [39]:
X7_down[:,-1], y7_down.sum(), len(y7_down)

(array([4.41566174e-04, 7.63336421e-02, 5.80971428e-02, 4.87392894e-02,
        1.95468770e-02, 6.09813178e-04, 1.12268157e-03, 1.97392913e-02,
        7.79977576e-05, 4.71912737e-03, 2.36503407e-03, 1.51403978e-05,
        1.34906608e-04, 6.10428266e-03, 2.93734562e-06, 2.61620934e-02,
        1.13646671e-03, 3.29906958e-03, 2.34578119e-04, 1.72961207e-02,
        2.86522566e-02, 4.06085158e-04, 6.02544089e-05, 2.32821381e-04,
        5.78783551e-03, 4.19685376e-04, 1.78107693e-05, 1.55285378e-03,
        2.09366581e-02, 1.93978139e-04, 1.04071050e-02, 8.85990520e-02,
        4.83017230e-04, 2.07975003e-04, 2.47503549e-03, 1.19947622e-04,
        7.37416666e-03, 3.58390504e-03, 5.88560210e-03, 1.31079274e-05,
        1.07971409e-03, 3.64658752e-03, 3.99212101e-04, 9.67768681e-03,
        9.32791627e-04, 1.09332981e-04, 1.04166585e-02, 2.04692629e-02,
        4.24821158e-03, 4.45834065e-04, 1.51472045e-04, 1.24655466e-02,
        2.24372251e-04, 2.93426354e-04, 1.26754889e-02, 2.249523

In [42]:
y7_down = y7_down.astype(int)

In [43]:
y7_down.shape, X7_down.shape, y7_down

((774,),
 (774, 920),
 array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0,
        0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0,
        1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0,

In [45]:
y_test

array([0, 0, 0, ..., 0, 0, 0])

In [44]:
clf7 = BalancedRandomForestClassifier(n_estimators=500, max_depth=5, random_state=0)
clf7.fit(X7_down, y7_down)  



BalancedRandomForestClassifier(bootstrap=True, class_weight=None,
                               criterion='gini', max_depth=5,
                               max_features='auto', max_leaf_nodes=None,
                               min_impurity_decrease=0.0, min_samples_leaf=2,
                               min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=500,
                               n_jobs=1, oob_score=False, random_state=0,
                               replacement=False, sampling_strategy='auto',
                               verbose=0, warm_start=False)

In [46]:
brfc_pred7 = clf7.predict(X_test)
brfc_cr7 = classification_report(y_test, brfc_pred7, digits=10)
print(brfc_cr7)

              precision    recall  f1-score   support

           0  0.9998969285 0.9295410265 0.9634362327     31309
           1  0.0178094390 0.9302325581 0.0349497597        43

    accuracy                      0.9295419750     31352
   macro avg  0.5088531837 0.9298867923 0.4991929962     31352
weighted avg  0.9985499725 0.9295419750 0.9621627918     31352



In [133]:
y_test.sum(), brfc_pred7.sum()

(111, 0.0)